In [1]:
import sqlite3 as sql
import numpy as np
from keras.preprocessing.text import Tokenizer

NUM_WORDS = 10000

data = sql.connect("movie_lines.db")
cursor = data.cursor()

Using TensorFlow backend.


In [2]:
cursor.execute("SELECT COUNT(movie_id) FROM movies")
count = cursor.fetchone()[0]

movie_lines = []
for i in range(count):
    cursor.execute("SELECT line_text FROM lines WHERE movie_id = {}".format(i))
    lines = ""
    lines = " ".join([line[0] for line in cursor.fetchall()])
    movie_lines.append(lines)
    

In [6]:
tokenizer = Tokenizer(num_words=NUM_WORDS, oov_token="<UNK>")
tokenizer.fit_on_texts(movie_lines)

In [13]:
features = tokenizer.texts_to_sequences(movie_lines)

In [16]:
def vectorize_sequences(sequences, dimension=10000):
    # Create an all-zero matrix of shape (len(sequences), dimension)
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.  # set specific indices of results[i] to 1s
    return results

x_train = vectorize_sequences(features)